In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00


In [ ]:
from pprint import pprint

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00


In [ ]:
!pip install -q evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00


In [ ]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from transformers import AutoConfig
from nltk.cluster.util import cosine_distance
from multiprocessing import Pool, cpu_count

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# read review file
datapath = '/content/drive/MyDrive/W266 Final Project/df_concatenated.csv'
top_review = pd.read_csv(datapath, header=0)
# select relevant columns only
# TODO: maybe we want to weigh "useful" more than "funny" when deciding top 10 reviews
text_col = 'text_top10'
cols = ['business_id', text_col]
top_review = top_review[cols]
top_review.shape

(10318, 2)

In [ ]:
# sample text for testing methods
text = top_review[text_col][2]

## Extractive Summarization - SpaCy

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [ ]:
# load the english pipeline model
nlp = spacy.load('en_core_web_sm')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
def get_keyword(doc):
  """ get keyword of the given text by filtering out stop words"""
  # create tokens
  keyword = []
  stopwords = list(STOP_WORDS)
  pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
  for token in doc:
    if (token.text in stopwords or token.text in punctuation):
      continue
    if (token.pos_ in pos_tag):
      keyword.append(token.text)

  return keyword

def count_frequencies(keyword):
  # count word frequencies
  freq_word = Counter(keyword)
  # normalize
  max_freq = max(freq_word.values())
  for word in freq_word.keys():
    freq_word[word] = freq_word[word]/max_freq
  return freq_word

def weigh_sentences(doc, freq_word):
  """ weigh sentences by summing word frequencies """
  sentence_scores = {}
  for sent in doc.sents:
    for word in sent:
      word = word.text.lower()
      if word in freq_word.keys():
        if sent not in sentence_scores.keys():
          sentence_scores[sent] = freq_word[word]
        else:
          sentence_scores[sent] += freq_word[word]
  return sentence_scores

def generate_spacy_summary(text, top_n=3):
  # parse text using nlp
  doc = nlp(text)
  keyword = get_keyword(doc)
  freq_word = count_frequencies(keyword)
  sentence_scores = weigh_sentences(doc, freq_word)
  # select top 5% of sentences to create a summary
  sentence_tokens = [sent for sent in doc.sents]
  #select_length = int(len(sentence_tokens)*sample)
  summarized_sentences = nlargest(top_n, sentence_scores, key=sentence_scores.get)
  # concatenate to create final summary
  final_sentences = [word.text for word in summarized_sentences]
  summary = ' '.join(final_sentences)
  return summary

In [ ]:
generate_spacy_summary(text)

"I'd really love to come back to try their dinner service, but we were at the bar for HAPPY HOUR\n\nbar snacks and cocktails for the happy hour menu\n---indebluetini: refreshing mango drink, loved the cardamom\n---spinach chaat: the reason my friend suggested indeblue, DELICIOUS flash fried spinach!\n We tried -\n\n1) Crab Dosa and Fish Tacos for Apps\n2) Shrimp Curry and Panner Masala, with assorted naans as entrees\n3) Kulfi, Chocolate Mousse with tea as the last course\n\nWe really liked the flavor profile of our food, everything was really good but Crab dosa was my favorite, though we were hesitant to try dosa with seafood filing but it turned out to be really delicious. We ordered: crispy cauliflower, vegetable samosas, crispy spinach chaat (a longtime favorite!), sea bass entree, shrimp moilee, paneer tikka masala, and a double order of naan."

In [ ]:
# apply to all of pandas dataframe
top_review['spacy_summary'] = top_review['text_top10'].apply(lambda x: generate_spacy_summary(x, top_n=3))
top_review.shape

(10318, 3)

In [ ]:
top_review.to_csv('/content/drive/MyDrive/W266 Final Project/spacy_summary.csv', index = False)
print('spacy summary completed!')

spacy summary completed!


## Extractive Summarization - TextRank

In [ ]:
def sentence_similarity(sent1, sent2, stopwords=None):
  """ find cosine similarity between two sentences """
  words1 = [w.lower() for w in sent1]
  words2 = [w.lower() for w in sent2]
  all_words = list(set(words1 + words2))

  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)

  # Build vectors representing each sentence
  for w in words1:
      if w not in stopwords:
          vector1[all_words.index(w)] += 1

  for w in words2:
      if w not in stopwords:
          vector2[all_words.index(w)] += 1

  return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stopwords):
  """ build similarity matrix between two sentences using sentence similarity """
  similarity_matrix = np.zeros((len(sentences), len(sentences)))
  for i in range(len(sentences)):
      for j in range(len(sentences)):
          if i != j:
              similarity_matrix[i][j] = sentence_similarity(sentences[i], sentences[j], stopwords)
  return similarity_matrix

def generate_textrank_summary(review, top_n=5):
  """ generate summary using cosine similarity """
  summarize_text = []

  sentences = nltk.sent_tokenize(review)
  clean_sentences = [nltk.word_tokenize(sent) for sent in sentences]
  sentence_similarity_matrix = build_similarity_matrix(clean_sentences, STOP_WORDS)

  # Apply PageRank algorithm
  scores = np.sum(sentence_similarity_matrix, axis=1)
  ranked_sentences = [sentences[i] for i in np.argsort(scores)[-top_n:]]
  summarize_text.extend(ranked_sentences)

  return ' '.join(summarize_text)

def generate_textrank_summary_batch(chunk):
  """ creates batches of input """
  summaries = []
  for review in chunk:
      summary = generate_textrank_summary(review, top_n=3)
      summaries.append(summary)
  return summaries

def main_textrank_summary(df):
  """ optimize performance by running in batches """
  num_processes = cpu_count() - 1  # Use all available CPU cores except one
  chunk_size = 1000
  with Pool(processes=num_processes) as pool:
      chunks = [df["text_top10"][i:i + chunk_size] for i in range(0, len(df), chunk_size)]
      results = pool.map(generate_textrank_summary_batch, chunks)

  # Flatten the list of results and assign it to a new "summary" column
  df["textrank_summary"] = [summary for chunk in results for summary in chunk]
  return df

In [ ]:
%%time
top_review = main_textrank_summary(top_review)
top_review.shape

/usr/local/lib/python3.10/dist-packages/nltk/cluster/util.py:130: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


CPU times: user 44.3 s, sys: 5.73 s, total: 50 s
Wall time: 2h 22min 52s


(10318, 4)

In [ ]:
top_review.to_csv('/content/drive/MyDrive/W266 Final Project/textrank_summary.csv', index = False)
print('textrank summary completed!')

textrank summary completed!


## Abstractive Summarization - Pegasus/cnn_dailymail

In [ ]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

cnnmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)
cnntokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
config = AutoConfig.from_pretrained("google/pegasus-cnn_dailymail")

In [ ]:
ARTICLE_TO_SUMMARIZE = (
    top_review['text_top10'][2]
)

LONG_REFERENCE = (
    "IndeBlue is a delightful restaurant with a beautiful interior and attentive staff."
    "The must-try dishes include the super yummy and crispy spinach chaat, as well as the spicy chicken and waffles."
    "Their chef's tasting menu is perfect for larger groups, offering a variety of delicious Indian fusion dishes."
    "The bar also impresses with refreshing drinks like the indebluetini and the mango-flavored martini."
    "Overall, IndeBlue is highly recommended for a sophisticated and wonderful dining experience, whether for a date night or a gathering with friends and family."
)

In [ ]:
cnninputs = cnntokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="tf")

In [ ]:
%%time

# Generate Summary
summary_ids = cnnmodel.generate(cnninputs["input_ids"],
    num_beams = 4,
    min_length = 50,
    no_repeat_ngram_size = 5,
    max_new_tokens = 200
)

candidate = cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
pprint(candidate[0], compact=True)

('IndeBlue offers a four-course tasting menu .<n>Try the spinach chaat for '
 'lunch or the spicy chicken and waffles for brunch .<n>The restaurant remains '
 'spotless the 1 1/2 hours I was there .<n>The tasting menu comes with 4 '
 'courses .')
CPU times: user 2min 4s, sys: 10.9 s, total: 2min 15s
Wall time: 1min 54s


In [ ]:
rouge = evaluate.load('rouge')
predictions = candidate
references = [LONG_REFERENCE]
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.3181818181818182, 'rouge2': 0.0923076923076923, 'rougeL': 0.2424242424242424, 'rougeLsum': 0.2424242424242424}


## Abstractive Summarization - BART/large-cnn

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the BART tokenizer and model
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
#device = torch.device('cpu')

# Tokenize the input text and generate the summary
inputs = bart_tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="pt")
summary_ids = bart_model.generate(inputs["input_ids"],
    num_beams=4,
    min_length=50,
    no_repeat_ngram_size=5,
    max_length=200,  # Adjust the max_length based on your preference
    early_stopping=True)

# Decode the generated summary
summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

IndeBlue offers a chef's tasting menu with 4 courses and is reasonably priced. Everything is served family style and you get to hand select several different food entrees for each course. If you are apprehensive about trying Indian food, this restaurant is the best place to try the new cuisine.


In [ ]:
summary

"IndeBlue offers a chef's tasting menu with 4 courses and is reasonably priced. Everything is served family style and you get to hand select several different food entrees for each course. If you are apprehensive about trying Indian food, this restaurant is the best place to try the new cuisine."

In [ ]:
LONG_REFERENCE

"IndeBlue is a delightful restaurant with a beautiful interior and attentive staff.The must-try dishes include the super yummy and crispy spinach chaat, as well as the spicy chicken and waffles.Their chef's tasting menu is perfect for larger groups, offering a variety of delicious Indian fusion dishes.The bar also impresses with refreshing drinks like the indebluetini and the mango-flavored martini.Overall, IndeBlue is highly recommended for a sophisticated and wonderful dining experience, whether for a date night or a gathering with friends and family."

In [ ]:
rouge = evaluate.load('rouge')
predictions = [summary]
references = [LONG_REFERENCE]
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.2714285714285714, 'rouge2': 0.043478260869565216, 'rougeL': 0.15714285714285714, 'rougeLsum': 0.15714285714285714}
